# Hypergrams
[Plotly express](https://plotly.com/python/plotly-express/) makes it pretty straightforward to create HTML [hypergrams](https://worrydream.com/refs/Nelson_T_1974_-_Computer_Lib,_Dream_Machines.pdf). These can then be shared by email and opened locally in a browser, or embedded in a webpage. These aren't really relevant to OpenSpace (unless you want to open them in an embedded browser window) but they might come in handy if you want a lightweight interactive visualization.

In [ ]:
import plotly.express as px

from datetime import datetime as dt
import pandas as pd
import openspace_rvdata.r2r2df as r2r
import numpy as np

import logging
logger = logging.getLogger(__name__)

## Ship Track Visualization 
<!-- Are you reading [this book](https://milkweed.org/book/the-quickening) and trying to sort out what's happening when? That was [NBP1902](https://www.rvdata.us/search/cruise/NBP1902). I'd sure like to make that the example, but it's not available as a geoCSV. Oh well.  -->
We can use RR2402 as our example.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import io

# Download file:
r2r.get_cruise_nav("RR2402")

# Read the CSV data directly from the file.
# The 'comment='#' will automatically skip lines starting with '#'.
df = pd.read_csv("tmp/RR2402_control.geoCSV", comment='#')

# Convert 'iso_time' column to datetime objects
df['iso_time'] = pd.to_datetime(df['iso_time'])

# Sort the DataFrame by time to ensure correct animation order
df = df.sort_values(by='iso_time')

# Add a 'trace_id' column for the animation_group.
# By making this a constant, Plotly Express will draw the entire trace
# up to the current animation frame, creating a persistent trail.
df['trace_id'] = 'ship_track'

# Create the animated geoscatter plot using plotly express
# 'lon' for longitude, 'lat' for latitude
# 'animation_frame' is set to 'iso_time' to create the animation
# 'animation_group' is set to 'trace_id' to make the line persist
# 'title' provides a descriptive title for the plot
# 'height' sets the height of the plot
# 'size_max' adjusts the marker size for the current point
# 'projection' sets the map projection type
fig = px.scatter_geo(df,
                     lon="ship_longitude",
                     lat="ship_latitude",
                    #  color="speed_made_good",
                     animation_frame="iso_time",
                     animation_group="trace_id", # This makes the line persist
                     title="RR2402 Ship Track",
                     height=600,
                     size_max=500,
                     projection="natural earth", # Or "orthographic", "mercator", "winkel3", etc.
                     labels={"ship_longitude": "Longitude", "ship_latitude": "Latitude", "iso_time": "Time"}
                    )
# Create line trace
line_trace = go.Scattergeo(lat=df['ship_latitude'], lon=df['ship_longitude'], mode='lines', name='line')
# Add line trace to the figure
fig.add_trace(line_trace)
fig.update_traces(line=dict(width=2)) # Set the line width to 2 (adjust as needed)


# ---
## Centering the Map
# ---

# Update the geographic scope to center the map over Southern California
fig.update_geos(
    center=dict(lat=33.5, lon=-118),  # Approximate center of Southern California
    lataxis_range=[30, 37],  # Latitude range for Southern California
    lonaxis_range=[-124, -114], # Longitude range for Southern California
    landcolor="rgb(243, 243, 243)",
    countrycolor="rgb(204, 204, 204)"
)

# ---
## Adjusting Animation Speed
# ---

# Update layout for better aesthetics, including adjusting the animation speed
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 500  # Adjust animation speed in milliseconds
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 200 # Adjust transition duration

# Show the plot
fig.update_layout(showlegend=False)
fig.show()
fig.write_html('plots/RR2402.html')

## Gantt Chart: Multiple Cruises in Context

In [ ]:
# mdf[mdf.has_r2rnav == False].is_retrieved
import plotly.express as px
title = "Cruises"

# url = r2r.get_r2r_url(vessel_name = "Palmer")
url = "https://service.rvdata.us/api/cruise/"
mdf = r2r.get_cruise_metadata(url)
mdf['waterbody_name'] = mdf['waterbody_name'].fillna("Not listed")
# mdf['waterbody_name'] = mdf['waterbody_name'].replace("null", "Not listed")  # Replace all 2s with 10 in column 'A'

mdf.head()

In [ ]:
mdf.columns

In [ ]:
title = "RV Cruises by Water Body"

# TODO: add absolute lat column latitude_abs = max(abs(latitude_max), abs(latitude_min))

# Wrap the text and replace newlines with <br>
mdf['wrapped_text'] = mdf['cruise_name'].fillna('').str.wrap(30).apply(lambda x: x.replace('\n', '<br>'))
mdf["cruise_name"] = mdf["wrapped_text"]

fig = px.timeline(
    mdf,
    x_start="depart_date",
    x_end="arrive_date",
    y="vessel_shortname",  # This will be the label for each bar on the y-axis
    color="waterbody_name", # You can color by the same ID or another categorical column
    color_continuous_scale=px.colors.sequential.RdBu, # use RdBu divergent colormap to highlight polar cruises
    title=title,
    # Other optional arguments:
    hover_name="cruise_name", # Show more info on hover
    hover_data=["cruise_id", "waterbody_name", "cruise_doi"], # Show more info on hover
    # text="label_on_bar", # Text directly on the bar
    # facet_row="some_category", # For breaking down by a category
    # template="plotly_white" # or "plotly_dark", "ggplot2", "seaborn", etc.
    template = "seaborn"
)

# You might want to sort the y-axis (tasks) for better readability
fig.update_yaxes(autorange="reversed") # To display the first task at the top

fig.show()
fig.write_html("plots/RV Cruises by Waterbody.html")

In [ ]:
title = "RV Cruises by Data Availability"

# TODO: add absolute lat column latitude_abs = max(abs(latitude_max), abs(latitude_min))

# Wrap the text and replace newlines with <br>
mdf['wrapped_text'] = mdf['cruise_name'].fillna('').str.wrap(30).apply(lambda x: x.replace('\n', '<br>'))
mdf["cruise_name"] = mdf["wrapped_text"]

fig = px.timeline(
    mdf,
    x_start="depart_date",
    x_end="arrive_date",
    y="vessel_shortname",  # This will be the label for each bar on the y-axis
    color="has_r2rnav", # You can color by the same ID or another categorical column
    # color_continuous_scale=px.colors.sequential.RdBu, # use RdBu divergent colormap to highlight polar cruises
    title=title,
    # Other optional arguments:
    hover_name="cruise_name", # Show more info on hover
    hover_data=["cruise_id", "waterbody_name", "cruise_doi"], # Show more info on hover
    # text="label_on_bar", # Text directly on the bar
    # facet_row="some_category", # For breaking down by a category
    # template="plotly_white" # or "plotly_dark", "ggplot2", "seaborn", etc.
    template = "seaborn"
)

# You might want to sort the y-axis (tasks) for better readability
fig.update_yaxes(autorange="reversed") # To display the first task at the top

fig.show()
fig.write_html("plots/RV Cruises by Data Availability.html")

Now let's look just at polar cruises. 

TODO: R2R doesn't have position information listed for all of these. See if there's a way to extract it from the data and populate that part of the dataframe, or ask them to fix it. 

In [ ]:

# restrict to only polar cruises (use min and max lat)
mdf["is_arctic"]= mdf['latitude_max'] > 65
mdf["is_antarctic"]= mdf['latitude_min'] < -60

mdf['Furthest Latitude'] = np.where(
    mdf['latitude_max'].abs() > mdf['latitude_min'].abs(),
    mdf['latitude_max'],
    mdf['latitude_min']
)

mdf_arctic = mdf[mdf.is_arctic == True]
mdf_antarctic = mdf[mdf.is_antarctic == True]
mdf_polar = mdf[(mdf['is_arctic'] == True) | (mdf['is_antarctic'] == True)]

title = "Arctic Cruises"
fig = px.timeline(
    mdf_arctic,
    x_start="depart_date",
    x_end="arrive_date",
    y="vessel_shortname",  # This will be the label for each bar on the y-axis
    color="latitude_max", # You can color by the same ID or another categorical column
    title=title,
    # Other optional arguments:
    hover_name="cruise_name", # Show more info on hover
    hover_data=["cruise_id", "waterbody_name", "cruise_doi"], # Show more info on hover
    # text="label_on_bar", # Text directly on the bar
    # facet_row="some_category", # For breaking down by a category
    # template="plotly_white" # or "plotly_dark", "ggplot2", "seaborn", etc.
    template = "seaborn",
    labels={"vessel_shortname": "Research Vessel"}
)

# You might want to sort the y-axis (tasks) for better readability
# fig.update_yaxes(autorange="reversed") # To display the first task at the top

fig.show()
fig.write_html("plots/Arctic Cruises.html")


title = "Antarctic Cruises"
fig = px.timeline(
    mdf_antarctic,
    x_start="depart_date",
    x_end="arrive_date",
    y="vessel_shortname",  # This will be the label for each bar on the y-axis
    color="latitude_min", # You can color by the same ID or another categorical column
    title=title,
    # Other optional arguments:
    hover_name="cruise_name", # Show more info on hover
    hover_data=["cruise_id", "waterbody_name", "cruise_doi"], # Show more info on hover
    # text="label_on_bar", # Text directly on the bar
    # facet_row="some_category", # For breaking down by a category
    # template="plotly_white" # or "plotly_dark", "ggplot2", "seaborn", etc.
    template = "seaborn",
    labels={"vessel_shortname": "Research Vessel"}
)

# You might want to sort the y-axis (tasks) for better readability
# fig.update_yaxes(autorange="reversed") # To display the first task at the top

fig.show()
fig.write_html("plots/Antarctic Cruises.html")


title = "Polar Cruises"
fig = px.timeline(
    mdf_polar,
    x_start="depart_date",
    x_end="arrive_date",
    y="vessel_shortname",  # This will be the label for each bar on the y-axis
    color="Furthest Latitude", # You can color by the same ID or another categorical column
    color_continuous_scale=px.colors.sequential.RdBu_r,
    title=title,
    # Other optional arguments:
    hover_name="cruise_name", # Show more info on hover
    hover_data=["cruise_id", "waterbody_name", "cruise_doi"], # Show more info on hover
    # text="label_on_bar", # Text directly on the bar
    # facet_row="some_category", # For breaking down by a category
    # template="plotly_white" # or "plotly_dark", "ggplot2", "seaborn", etc.
    template = "seaborn", 
    labels={"vessel_shortname": "Research Vessel", "Furthest Latitude":"Latitude"}
)

# You might want to sort the y-axis (tasks) for better readability
# fig.update_yaxes(autorange="reversed") # To display the first task at the top

fig.show()
fig.write_html("plots/Polar Cruises.html")


### Gantt chart, filtered by keyword
Suppose we want to check all the cruises that relate to a given topic. Here's some example code for that.

In [ ]:
# The keyword you want to filter by
keyword_to_find = "LTER"

# Filter the DataFrame
# filtered_mdf = mdf[mdf['keyword_list'].apply(lambda x: keyword_to_find in x)]
filtered_cruises = mdf[mdf['cruise_name'].str.contains(keyword_to_find, case=True, na=False)]
# filtered_cruises


fig = px.timeline(
    filtered_cruises,
    x_start="depart_date",
    x_end="arrive_date",
    y="vessel_shortname",  # This will be the label for each bar on the y-axis
    color="waterbody_name", # You can color by the same ID or another categorical column
    title=keyword_to_find + " Cruises",
    # Other optional arguments:
    hover_name="wrapped_text", # Show more info on hover
    hover_data=["cruise_id", "waterbody_name", "cruise_doi"], # Show more info on hover
    # text="label_on_bar", # Text directly on the bar
    # facet_row="some_category", # For breaking down by a category
    # template="plotly_white" # or "plotly_dark", "ggplot2", "seaborn", etc.
    template = "seaborn"
)

# You might want to sort the y-axis (tasks) for better readability
fig.update_yaxes(autorange="reversed") # To display the first task at the top

fig.show()
fig.write_html("plots/" + keyword_to_find + " Cruises.html")